In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行的端到端机器学习：MLOps阶段4：评估：开始使用Vertex AI模型评估

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage4/get_started_with_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage4/get_started_with_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage4/get_started_with_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示了如何在谷歌云上使用Vertex AI进行端到端MLOps生产过程。此教程涵盖了第4阶段：评估：开始使用Vertex AI模型评估。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Model Evaluation`。

本教程使用以下谷歌云机器学习服务：

- `Vertex AI AutoML`
- `BigQuery ML`
- `Vertex AI Training`
- `Vertex AI Batch Prediction`
- `Vertex AI Model Evaluation`
- `Google Cloud Pipeline Components`

执行的步骤包括：

**SDK**

- 评估一个 `AutoML` 模型。
    - 训练一个 `AutoML` 图像分类模型。
    - 获取来自训练的默认评估指标。
    - 为自定义的评估切片执行批量评估。
- 评估一个 BigQuery ML 模型。
    - 训练一个 `BigQuery ML` 表格分类模型。
    - 获取来自训练的默认评估指标。
    - 为自定义的评估切片执行批量评估。
- 评估一个自定义模型。
    - 为自定义的评估切片执行批量评估。
    - 将一个评估添加到 `Model Registry` 中的 `Model` 资源。

**管道组件**

- 评估一个 `AutoML` 模型。
    - 训练一个 `AutoML` 图像分类模型。
    - 获取来自训练的默认评估指标。
    - 为自定义的评估切片执行批量评估。
- 评估一个 BigQuery ML 模型。
    - 训练一个 `BigQuery ML` 表格分类模型。
    - 获取来自训练的默认评估指标。
    - 为自定义的评估切片执行批量评估。
- 评估一个自定义模型。
    - 为自定义的评估切片执行批量评估。
    - 为 `Model` 资源的 `Model Registry` 添加一个评估。

### 数据集

**AutoML 图像模型**

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[Flowers数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。本教程中的数据集版本存储在一个公共Cloud Storage存储桶中。训练好的模型可以预测图像属于五种不同花卉中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

**BigQuery ML 表格模型**

本教程中使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)中的企鹅数据集。该数据集版本用于根据可用特征如喙长、鳍深度等预测企鹅的物种。

**自定义模型**

本教程使用了一个在ImageNet数据集上训练的来自TensorFlow Hub的预训练图像分类模型。

了解有关[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)的更多信息。

**流水线**

本教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)。这个数据集不需要任何特征工程处理。本教程中的数据集版本存储在一个公共Cloud Storage存储桶中。

费用
此教程使用Google Cloud的计费组件：

- Vertex AI
- 云存储
- BigQuery

了解有关[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，[云存储定价](https://cloud.google.com/storage/pricing)和[BigQuery定价](https://cloud.google.com/bigquery/pricing)的信息，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 安装

安装本笔记本执行所需的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install --upgrade google-cloud-bigquery $USER_FLAG -q
! pip3 install --upgrade tensorflow $USER_FLAG -q
! pip3 install --upgrade tensorflow-hub $USER_FLAG -q

### 重新启动内核

一旦你安装了额外的包，你需要重新启动笔记本内核，这样它就能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果您有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用结算。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter在以`!`为前缀的行上运行作为shell命令，并会解析以`$`为前缀的Python变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改 `REGION` 变量，此变量用于本笔记本的其余操作。以下是 Vertex AI 支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶进行 Vertex AI 训练。并非所有地区都支持所有 Vertex AI 服务。

了解有关[Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加直播教程会议，您可能会使用共享测试账户或项目。为了避免用户在创建的资源之间发生名称碰撞，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用Vertex AI工作台笔记本**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，按照以下步骤操作：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"存储对象管理员"，然后选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件会下载到您的本地环境。

在下面的单元格中输入您的服务账户密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储存储桶

**无论您使用的是什么笔记本环境，下面的步骤都是必要的。**

当您初始化 Python 的 Vertex SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是所有与您的数据集和模型资源相关的数据在会话之间保留的地方。

在下面设置您的云存储存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您的组织外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时：运行以下代码以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容验证您的云存储桶的访问权限：

In [ ]:
! gsutil ls -al $BUCKET_URI

#### 服务账户

**如果您不知道您的服务账户**，请尝试使用`gcloud`命令在下面执行第二个单元格以获取您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置 Vertex AI Pipelines 的服务账号访问权限

运行以下命令，为您的服务账号授予读取和写入流水线文件的权限，该文件位于您在上一步创建的存储桶中--您只需要为每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库并定义常量

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import tensorflow as tf
import tensorflow_hub as hub
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component

#### 导入BigQuery

在您的Python环境中导入BigQuery包。

In [ ]:
from google.cloud import bigquery

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建BigQuery客户端。

In [ ]:
bqclient = bigquery.Client()

#### 设置硬件加速器

您可以为预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，并为每个VM分配4个Nvidia Telsa K80 GPU，则应指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`来使用一个在CPU上运行的容器映像。

在[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)了解更多关于您地区的硬件加速器支持。

In [ ]:
import os

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

设置预构建的容器

为预测设置预构建的Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示一些可用的预构建镜像：

有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPU`：\[2, 4, 8, 16, 32, 64, 96 \]中的数字

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## Vertex AI模型评估介绍，适用于AutoML模型。

对于AutoML模型，您可以使用`Vertex AI模型评估`服务从划分为训练集和测试集的数据集中检索在训练过程中获得的模型评估指标。此外，您还可以使用自定义评估切片进一步评估模型。

云存储培训数据的位置。

现在将变量“IMPORT_FILE”设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法为`Dataset`资源创建数据集，需要传入以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Flowers" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练AutoML模型，您需要执行两个步骤：1）创建训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建AutoML训练管道，参数如下：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：要为其训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则是单标签(`False`)还是多标签(`True`)。
- `model_type`：部署模型的类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：针对部署在Google Cloud上的准确性优化。
  - `CLOUD_LOW_LATENCY`：针对部署在Google Cloud上的延迟优化。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：针对部署在边缘设备上的准确性优化。
  - `MOBILE_TF_LOW_LATENCY_1`：针对部署在边缘设备上的延迟优化。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习 -- 仅支持图像分类。

实例化的对象是训练作业的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 运行训练管道

接下来，您可以运行DAG以通过调用`run`方法来启动训练作业，参数如下：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留存数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: （可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`: 如果为`True`，则训练可能在服务认为无法进一步提高模型客观度量之前完成。

当`run`方法完成时，会返回`Model`资源。

执行训练管道将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

### 获取`AutoML Model`资源的默认评估

BLAH GAPIC 占位符
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量引用，或者列出项目中的所有模型。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

### 在自定义评估切片上评估

占位符 - 废话

### 创建批量输入文件

现在创建一个批量输入文件，将其存储在您的本地云存储存储桶中。批量输入文件必须采用JSONL格式。对于JSONL文件，您需要为每个数据项（实例）的每一行创建一个字典条目。该字典包含键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的示例中，它是一个`jpeg`文件。

例如：

                        {'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}
    
为了演示目的，创建一个评估切片，您可以使用部分训练数据 -- 就好像它是单独的（非训练）数据，例如在生产环境中看到的实例。

In [ ]:
EVAL_SLICE = BUCKET_URI + "/flowers_eval.jsonl"

! gsutil cat {IMPORT_FILE} | head -n 200 >tmp.csv

import csv

entries = []
with open("tmp.csv", "r") as f:
    reader = csv.reader(f)
    for row in reader:
        path = row[0]
        label = row[1]

        file = path.split("/")[-1]

        new_path = BUCKET_URI + "/flowers/" + file

        ! gsutil cp {path} {new_path} >/dev/null
        entries.append({"content": new_path, "mime_type": "jpeg"})

import json

with open("tmp.jsonl", "w") as f:
    for entry in entries:
        f.write(json.dumps(entry) + "\n")

! gsutil cp tmp.jsonl {EVAL_SLICE}
#! rm tmp.csv tmp.jsonl

### 发起批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测工作的可读名称。
- `instances_format`：预测请求的格式；只能是JSONL（默认）。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，则调用将在等待异步批量作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="flowers_" + TIMESTAMP,
    instances_format="jsonl",
    gcs_source=EVAL_SLICE,
    gcs_destination_prefix=BUCKET_URI,
    sync=True,
)

print(batch_predict_job)

### TODO: 获取批量结果

获得预测

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法来获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于0和1之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

#### 删除临时资源

接下来，您删除此示例创建的所有临时资源。

In [ ]:
try:
    dag.delete()
    model.delete()
    batch_predict_job.delete()
except Exception as e:
    print(e)

## 介绍如何使用 Vertex AI 模型评估来评估 BigQuery ML 模型。

对于 BigQuery ML 模型，您可以使用 `Vertex AI 模型评估` 服务从在训练期间获取的数据集分割为训练和测试的模型评估指标。此外，您还可以使用自定义评估分片进一步评估模型。

### BigQuery培训数据的位置

现在将变量`IMPORT_FILE`和`BQ_TABLE`设置为`BigQuery`中培训数据的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### 创建BQ数据集资源

首先，在您的项目中创建一个空的数据集资源。

In [ ]:
BQ_DATASET_NAME = "penguins"
DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}
"""

job = bqclient.query(DATASET_QUERY)

### 设置权限以自动注册模型

在训练后自动上传和注册模型之前，您需要为BigQuery ML设置一些额外的IAM权限。根据您的服务帐号，设置下面的权限可能会失败。在这种情况下，我们建议在Cloud Shell中执行权限。

In [ ]:
! gcloud projects add-iam-policy-binding $PROJECT_ID \
      --member='serviceAccount:cloud-dataengine@system.gserviceaccount.com' \
      --role='roles/aiplatform.admin'

! gcloud projects add-iam-policy-binding $PROJECT_ID \
      --member='user:cloud-dataengine@prod.google.com' \
      --role='roles/aiplatform.admin'

### 训练和注册BigQuery ML模型

接下来，您将从公共数据集企鹅中创建并训练一个BigQuery ML表格分类模型，并使用`CREATE MODEL`语句将模型存储在您的项目中。模型配置在`OPTIONS`语句中指定如下：

- `model_type`：要训练的表格模型的类型和架构，例如DNN分类。
- `labels`：标签所在的列。
- `model_registry`：设置为"vertex_ai"，表示自动注册到`Vertex AI Model Registry`。
- `vertex_ai_model_id`：已注册模型的人类可读的显示名称。
- `vertex_ai_model_version_aliases`：模型的备用名称。

了解更多关于[CREATE MODEL语句](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)。

In [ ]:
MODEL_NAME = "penguins"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='DNN_CLASSIFIER',
    labels = ['species'],
    model_registry="vertex_ai",
    vertex_ai_model_id="bqml_model_{TIMESTAMP}", 
    vertex_ai_model_version_aliases=["1"]
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

job = bqclient.query(MODEL_QUERY)
print(job.errors, job.state)

while job.running():
    from time import sleep

    sleep(30)
    print("Running ...")
print(job.errors, job.state)

tblname = job.ddl_target_table
tblname = "{}.{}".format(tblname.dataset_id, tblname.table_id)
print("{} created in {}".format(tblname, job.ended - job.started))

### 使用BigQuery评估已训练的BigQuery模型

接下来，在BigQuery中检索已训练的BigQuery ML模型的模型评估。

了解更多关于[ML.EVALUATE函数](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate)。

In [ ]:
EVAL_QUERY = f"""
SELECT *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{MODEL_NAME})
ORDER BY  roc_auc desc
LIMIT 1"""

job = bqclient.query(EVAL_QUERY)
results = job.result().to_dataframe()
print(results)

### 在`Vertex AI模型注册表`中找到模型

最后，您可以使用`Vertex AI模型`的list()方法并带有过滤查询来找到自动注册的模型。

In [ ]:
models = aiplatform.Model.list(filter="display_name=bqml_model_" + TIMESTAMP)
model = models[0]

print(model.gca_resource)

### 从 `Vertex AI Model Registry` 中检索`BigQuery ML Model`资源的默认评估

BLAH GAPIC 占位符
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=model.resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

### 在自定义评估片段上进行评估

占位符 - 垃圾

垃圾 - 批处理文件格式

#### 删除临时资源

接下来，删除该示例创建的所有临时资源。

In [ ]:
try:
    model.delete()
    batch_predict_job.delete()
except Exception as e:
    print(e)

try:
    # Delete the created BigQuery dataset
    ! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME
except Exception as e:
    print(e)

MODEL_QUERY = f"""
DROP MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
"""

job = bqclient.query(MODEL_QUERY)

## 介绍自定义模型的Vertex AI模型评估。

对于自定义模型，您可以使用`Vertex AI模型评估`服务从训练集中提取在训练期间获取的模型评估指标。此外，您还可以使用自定义评估切片进一步评估模型。

## 从TensorFlow Hub获取预训练模型

出于演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练模型，然后将其上传到`Vertex AI Model`资源。一旦您拥有了`Vertex AI Model`资源，该模型就可以部署到`Vertex AI Endpoint`资源中。

### 下载预训练模型

首先，您需要从TensorFlow Hub下载预训练模型。该模型将作为TF.Keras层进行下载。在本示例中，为了完成模型，您将创建一个包含下载的TFHub模型作为层的`Sequential()`模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

### 保存模型工件

此时，模型位于内存中。接下来，您需要将模型工件保存到一个云存储位置。

*注意:* 对于 TF Serving，MODEL_DIR 必须以一个数字结尾的子文件夹，例如，1。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model/1"
tfhub_model.save(MODEL_DIR)

上传用于提供服务的模型

接下来，您将从自定义作业中将您的TF.Keras模型上传到Vertex `Model`服务，该服务将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个用于将数据转换为您的模型期望格式的服务函数。如果您将编码数据发送到Vertex AI，您的服务函数将确保在将数据作为输入传递到您的模型之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将由HTTP服务器接收。HTTP服务器从HTTP请求内容主体中提取预测请求。提取的预测请求将转发到服务函数。对于Google预构建的预测容器，请求内容将以`tf.string`的形式传递给服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入(`tf.string`)转换为基础模型（动态图）期望的输入形状和数据类型。
  - 执行与培训基础模型期间相同的数据预处理操作--例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序期望的格式--例如，压缩输出。
  - 封装输出以供接收应用程序使用--例如，添加标题，创建JSON对象等。

预处理和后处理函数都转换为与模型融合的静态图。基础模型的输出传递给后处理函数。后处理函数将转换/封装的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时需要考虑的一点是，它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果这样做，您将在服务函数的编译期间收到错误，指出您正在使用不受支持的EagerTensor。

###图像数据的提供功能

####预处理

要将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为base 64——这样可以使内容在通过网络传输二进制数据时免受修改。由于部署的模型期望输入数据为原始（未压缩）字节，因此您需要确保base 64编码的数据被转换回原始字节，然后经过预处理以符合模型输入要求，然后再将其作为输入传递到部署的模型。

要解决这个问题，您可以定义一个提供功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便将提供功能合并到底层模型中（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容被解码成一个Tensorflow字符串（`tf.string`），然后传递给提供功能（`serving_fn`）。提供功能将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`），以匹配模型的输入要求：

- `io.decode_jpeg`- 解压缩返回为带有三个通道（RGB）的Tensorflow张量的JPG图像。
- `image.convert_image_dtype`- 将整数像素值更改为浮点32，并在0和1之间重新调整像素数据。
- `image.resize`- 将图像调整大小以匹配模型的输入形状。

在这一点上，数据可以通过具体函数传递给模型（`m_call`）。提供功能是一个静态图，而模型是一个动态图。具体函数执行将输入数据从提供功能传递到模型的任务，并将来自模型的预测结果从模型重新传递到提供功能。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取服务函数签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要服务函数的签名。为什么？当我们将数据作为HTTP请求包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的服务函数将从base64转换为numpy数组。

在发起预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称 - 这将在您发起预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

将TensorFlow Hub模型上传到“Vertex AI Model”资源中。

最后，您将TFHub模型中的模型工件上传到“Vertex AI Model”资源中。

In [ ]:
model = aiplatform.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

### 翻译: 在自定义模型上进行批量预测

### BLAH 注册自定义评估指标

使用`Vertex AI Pipeline`组件进行模型评估

在这一部分中，您可以使用`Vertex AI Pipeline`组件对`AutoML`、`BigQuery ML`和自定义模型进行模型评估。

### AutoML模型评估管道组件

BLAH

另外，您可以使用BatchPredictionOp和ModelEvaluationOp组件组合来使用自定义评估切片评估AutoML模型，步骤如下：

- 自定义评估切片数据包含标签值（真实值）。
- 对自定义评估切片进行批量预测。
- 使用批量预测结果和标签值进行模型评估。

#### 云存储培训数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"
! gsutil cat {IMPORT_FILE} | head -n 40000 > train.csv
! gsutil cat {IMPORT_FILE} | head -n 1 >eval.csv
! gsutil cat {IMPORT_FILE} | tail -n 5200 >> eval.csv

IMPORT_TRAIN = BUCKET_NAME + "/train.csv"
IMPORT_EVAL = BUCKET_NAME + "/eval.csv"

! gsutil cp train.csv {IMPORT_TRAIN}
! gsutil cp eval.csv {IMPORT_EVAL}

! rm -f train.csv eval.csv

### 创建AutoML模型评估组件

Vertex AI预构建的管道组件目前没有用于检索AutoML模型评估的组件。因此，您将首先按照以下步骤编写自己的组件：

- 以AutoML训练组件返回的区域和模型工件作为输入。
- 创建一个客户端接口到Vertex AI模型服务（`metadata["resource_name"]）。
- 从模型工件参数构建模型的资源ID。
- 检索模型评估。
- 将模型评估作为字符串返回。

In [ ]:
from kfp.v2.dsl import Artifact, Input, Model


@component(packages_to_install=["google-cloud-aiplatform"])
def evaluateAutoMLModelOp(model: Input[Artifact], region: str) -> str:
    import logging

    import google.cloud.aiplatform.gapic as gapic

    # Get a reference to the Model Service client
    client_options = {"api_endpoint": f"{region}-aiplatform.googleapis.com"}
    model_service_client = gapic.ModelServiceClient(client_options=client_options)

    model_id = model.metadata["resourceName"]

    model_evaluations = model_service_client.list_model_evaluations(parent=model_id)
    model_evaluation = list(model_evaluations)[0]
    logging.info(model_evaluation)
    return str(model_evaluation)

### 构建AutoML训练的流水线，并进行批量模型评估

接下来，使用以下任务构建流水线:

- 创建一个Vertex AI数据集资源。
- 训练一个AutoML表格分类模型。
- 检索AutoML评估统计数据。
- 使用在训练过程中未使用的评估切片，对AutoML模型进行批量预测。
- 使用批量预测的结果对AutoML模型进行评估。

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/automl_lbn_training".format(BUCKET_NAME)


@dsl.pipeline(
    name="automl-lbn-training", description="AutoML tabular classification training"
)
def pipeline(
    import_file: str,
    batch_files: list,
    display_name: str,
    bucket: str = PIPELINE_ROOT,
    project: str = PROJECT_ID,
    region: str = REGION,
):
    from google_cloud_pipeline_components import aiplatform as gcc_aip
    from google_cloud_pipeline_components.experimental.evaluation import \
        ModelEvaluationOp
    from google_cloud_pipeline_components.v1.batch_predict_job import \
        ModelBatchPredictOp

    dataset_op = gcc_aip.TabularDatasetCreateOp(
        project=project, display_name=display_name, gcs_source=import_file
    )

    training_op = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name=display_name,
        optimization_prediction_type="classification",
        dataset=dataset_op.outputs["dataset"],
        model_display_name=display_name,
        training_fraction_split=0.8,
        validation_fraction_split=0.1,
        test_fraction_split=0.1,
        budget_milli_node_hours=8000,
        optimization_objective="minimize-log-loss",
        target_column="Deposit",
    )

    eval_op = evaluateAutoMLModelOp(model=training_op.outputs["model"], region=region)

    batch_op = ModelBatchPredictOp(
        project=project,
        job_display_name="batch_predict_job",
        model=training_op.outputs["model"],
        gcs_source_uris=batch_files,
        gcs_destination_output_uri_prefix=bucket,
        instances_format="csv",
        predictions_format="jsonl",
        model_parameters={},
        machine_type=DEPLOY_COMPUTE,
        starting_replica_count=1,
        max_replica_count=1,
    ).after(eval_op)

    batch_eval_op = ModelEvaluationOp(
        project=project,
        root_dir=bucket,
        problem_type="classification",
        classification_type="multiclass",
        ground_truth_column="Deposit",
        class_names=["0", "1"],
        predictions_format="jsonl",
        batch_prediction_job=batch_op.outputs["batchpredictionjob"],
    )

### 编译并执行AutoML训练和批量模型评估管道

接下来，您编译管道，然后执行它。管道接受以下参数，这些参数作为字典`parameter_values`传递：

- `import_file`：训练数据的Cloud Storage位置。
- `batch_files`：一个或多个评估数据的Cloud Storage位置列表。
- `display_name`：Vertex AI模型和端点资源的显示名称。
- `project`：项目ID。
- `region`：地区。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="automl_lbn_training.json"
)

pipeline = aip.PipelineJob(
    display_name="automl_lbn_training",
    template_path="automl_lbn_training.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "import_file": IMPORT_TRAIN,
        "batch_files": [IMPORT_EVAL],
        "display_name": "bank" + TIMESTAMP,
        "project": PROJECT_ID,
        "region": REGION,
    },
)

pipeline.run()

! rm -f automl_lbn_training.json

查看AutoML训练和批量评估管道的结果

In [ ]:
PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            break
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            break

    return EXECUTE_OUTPUT


print("tabular-dataset-create")
artifacts = print_pipeline_output(pipeline, "tabular-dataset-create")
print("\n\n")
print("automl-tabular-training-job")
artifacts = print_pipeline_output(pipeline, "automl-tabular-training-job")
print("\n\n")
print("evaluateautomlmodelop")
artifacts = print_pipeline_output(pipeline, "evaluateautomlmodelop")
output = !gsutil cat $artifacts
output = json.loads(output[0])
metrics = output["parameters"]["Output"]["stringValue"]
print("\n")
print(metrics)
print("\n\n")
print("model-batch-predict")
artifacts = print_pipeline_output(pipeline, "model-batch-predict")
output = !gsutil cat $artifacts
output = json.loads(output[0])
print("\n\n")
print(
    output["artifacts"]["batchpredictionjob"]["artifacts"][0]["metadata"][
        "gcsOutputDirectory"
    ]
)
print("model-evaluation")
artifacts = print_pipeline_output(pipeline, "model-evaluation")

删除管道作业

在管道作业完成之后，您可以使用`delete()`方法来删除管道作业。在完成之前，管道作业可以使用`cancel()`方法来取消。

In [ ]:
pipeline.delete()

## 为BigQuery ML模型介绍顶点AI模型评估。

对于BigQuery ML模型，您可以使用`BigQuery ML`服务从拆分为训练集和测试集的数据集中检索在训练期间获得的模型评估指标。

此外，您可以使用`BatchPredictionOp`和`ModelEvaluationOp`组件的组合来使用自定义评估切片评估BigQuery ML模型，如下所示：

- 自定义评估切片数据包含标签值（地面真相）。
- 在自定义评估切片上执行批量预测。
- 使用批量预测结果和标签值执行模型评估。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.ml_datasets.penguins"
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

In [ ]:
BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"
BQ_DATASET = BQ_TABLE.split(".")[1]


def get_data(slice_name, limit):
    query = f"""
    CREATE OR REPLACE TABLE `{slice_name}`
    AS (
        WITH
          penguins AS (
          SELECT
            island,
            sex,
            culmen_length_mm,
            culmen_depth_mm,
            flipper_length_mm,
            body_mass_g,
            species
          FROM
            `{BQ_TABLE}`
        )

        SELECT
          island,
          sex,
          culmen_length_mm,
          culmen_depth_mm,
          flipper_length_mm,
          body_mass_g,
          species
        FROM
          penguins
        LIMIT {limit}
    )
    """

    response = bqclient.query(query)
    _ = response.result()


BQ_TABLE_EVAL = f"{PROJECT_ID}.{BQ_DATASET}.penguins_eval"
IMPORT_EVAL = f"bq://{BQ_TABLE_EVAL}"
LIMIT = 44
get_data(BQ_TABLE_EVAL, LIMIT)

BQ_TABLE_TRAIN = f"{PROJECT_ID}.{BQ_DATASET}.penguins_train"
IMPORT_TRAIN = f"bq://{BQ_TABLE_TRAIN}"
LIMIT = "300 OFFSET 44"
get_data(BQ_TABLE_TRAIN, LIMIT)

### 构建用于 BigQuery ML 训练和批处理模型评估的流水线

接下来，构建以下任务的流水线：

- 创建一个 BigQuery ML 数据集资源。
- 训练一个 BigQuery ML 表格分类模型。
- 检索 BigQuery ML 评估统计数据。
- 使用在训练过程中未使用的评估切片，对 BigQuery ML 模型进行批量预测。
- 使用批量预测结果评估 BigQuery ML 模型。

In [ ]:
PIPELINE_ROOT = f"{BUCKET_NAME}/bq_query"


@dsl.pipeline(name="bq-hello-world", pipeline_root=PIPELINE_ROOT)
def pipeline(
    bq_train_table: str,
    bq_eval_table: str,
    label: str,
    class_names: list,
    dataset: str,
    model: str,
    artifact_uri: str,
    # num_trials: int,
    deploy_image: str,
    machine_type: str,
    min_replica_count: int,
    max_replica_count: int,
    display_name: str,
    bucket: str,
    accelerator_type: str = "",
    accelerator_count: int = 0,
    project: str = PROJECT_ID,
    location: str = "US",
    region: str = "us-central1",
):
    from google_cloud_pipeline_components.experimental.evaluation import \
        ModelEvaluationOp
    from google_cloud_pipeline_components.v1.batch_predict_job import \
        ModelBatchPredictOp
    from google_cloud_pipeline_components.v1.bigquery import (
        BigqueryCreateModelJobOp, BigqueryEvaluateModelJobOp,
        BigqueryExportModelJobOp, BigqueryQueryJobOp)
    from google_cloud_pipeline_components.v1.model import ModelUploadOp

    bq_dataset = BigqueryQueryJobOp(
        project=project, location="US", query=f"CREATE SCHEMA {dataset}"
    )

    bq_model = BigqueryCreateModelJobOp(
        project=project,
        location=location,
        query=f"CREATE OR REPLACE MODEL {dataset}.{model} OPTIONS (model_type='dnn_classifier', labels=['{label}']) AS SELECT * FROM `{bq_train_table}` WHERE body_mass_g IS NOT NULL AND sex IS NOT NULL",
    ).after(bq_dataset)

    bq_eval = BigqueryEvaluateModelJobOp(
        project=PROJECT_ID, location="US", model=bq_model.outputs["model"]
    ).after(bq_model)

    bq_export = BigqueryExportModelJobOp(
        project=project,
        location=location,
        model=bq_model.outputs["model"],
        model_destination_path=artifact_uri,
    ).after(bq_model)

    model_upload = ModelUploadOp(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=deploy_image,
        project=project,
        location=region,
    ).after(bq_export)

    batch_predict = ModelBatchPredictOp(
        project=project,
        job_display_name="batch_predict_job",
        model=model_upload.outputs["model"],
        bigquery_source_input_uri=bq_eval_table,
        bigquery_destination_output_uri=f"bq://{project}",
        instances_format="bigquery",
        predictions_format="bigquery",
        model_parameters={},
        machine_type=DEPLOY_COMPUTE,
        starting_replica_count=min_replica_count,
        max_replica_count=max_replica_count,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
    ).after(model_upload)

    batch_eval = ModelEvaluationOp(
        project=project,
        root_dir=bucket,
        problem_type="classification",
        classification_type="multiclass",
        ground_truth_column=label,
        class_names=class_names,
        predictions_format="jsonl",
        batch_prediction_job=batch_predict.outputs["batchpredictionjob"],
    )

### 编译和执行 BigQuery ML 训练和批量模型评估管道

接下来，您编译管道然后执行它。管道接受以下参数，这些参数作为字典 `parameter_values` 传递：

- `bq_train_table`: 包含训练数据的 BigQuery 表。
- `bq_eval_table`: 包含评估数据的 BigQuery 表。
- `label`: BigQuery 数据集的对应标签。
- `dataset`: BigQuery 数据集组件名称。
- `model`: BigQuery 模型组件名称。
- `artifact_uri`: 导出 BigQuery 模型工件的 Cloud Storage 位置。
- `num_trials`: 如果大于一，则使用 Vertex AI Vizier 服务执行指定数量的试验进行超参数调整。
- `deploy_image`: 用于提供预测服务的容器映像。
- `machine_type`: 提供预测服务的虚拟机。
- `min_replica_count`/`max_replica_count`: 用于自动扩展预测的虚拟机数量。
- `display_name`: Vertex AI Model 资源的显示名称。
- `project`: 项目 ID。
- `region`: 区域。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/bqmodel"

compiler.Compiler().compile(pipeline_func=pipeline, package_path="bqml.json")

pipeline = aip.PipelineJob(
    display_name="bqml",
    template_path="bqml.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "bq_train_table": BQ_TABLE_TRAIN,
        "bq_eval_table": IMPORT_EVAL,
        "label": "species",
        "class_names": [
            "Adelie Penguin (Pygoscelis adeliae)",
            "Chinstrap penguin (Pygoscelis antarctica)",
            "Gentoo penguin (Pygoscelis papua)",
        ],
        "dataset": "bqml_tutorial",
        "model": "penguins_model",
        "artifact_uri": MODEL_DIR,
        #'num_trials': 1,
        "deploy_image": DEPLOY_IMAGE,
        "display_name": "penguins",
        "machine_type": DEPLOY_COMPUTE,
        "min_replica_count": 1,
        "max_replica_count": 1,
        "accelerator_type": DEPLOY_GPU.name,
        "accelerator_count": 1,
        "bucket": BUCKET_NAME,
        "project": PROJECT_ID,
        "location": "US",
    },
    # enable_caching=False
)

pipeline.run()

! rm -rf bqml.json

查看BigQuery ML的训练和批量评估管道结果###

In [ ]:
PROJECT_NUMBER = pipeline.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            break
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            break

    return EXECUTE_OUTPUT


print("bigquery-query-job")
artifacts = print_pipeline_output(pipeline, "bigquery-query-job")
print("\n\n")
print("bigquery-create-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-create-model-job")
print("\n\n")
print("bigquery-evaluate-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-evaluate-model-job")
print("\n\n")
print("bigquery-export-model-job")
artifacts = print_pipeline_output(pipeline, "bigquery-export-model-job")
print("\n\n")
print("model-upload")
artifacts = print_pipeline_output(pipeline, "model-upload")
print("\n\n")
print("model-batch-predict")
artifacts = print_pipeline_output(pipeline, "model-batch-predict")
output = !gsutil cat $artifacts
output = json.loads(output[0])
print("\n\n")
print(
    output["artifacts"]["batchpredictionjob"]["artifacts"][0]["metadata"][
        "gcsOutputDirectory"
    ]
)
print("model-evaluation")
artifacts = print_pipeline_output(pipeline, "model-evaluation")

### 删除管道作业

在管道作业完成后，您可以使用`delete()`方法删除管道作业。在完成之前，可以使用`cancel()`方法取消管道作业。

In [ ]:
pipeline.delete()

删除BigQuery模型和数据集

接下来，删除BigQuery模型和数据集。

In [ ]:
try:
    job = bqclient.delete_model("bqml_tutorial.penguins_model")
except:
    pass
job = bqclient.delete_dataset("bqml_tutorial", delete_contents=True)

清理

要清理本项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

- 数据集
- 流水线
- 模型
- 端点
- AutoML 训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.undeploy_all()
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline training job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom training job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME